In [2]:
# we are going to perform single training step - forward and then backward.
import torch
from torchvision.models import resnet18, ResNet18_Weights

model = resnet18(weights=ResNet18_Weights.DEFAULT)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /home/ec2-user/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 175MB/s]


In [3]:
# forward pass
prediction = model(data)

In [4]:
# calculate loss
# backpropagate this error
loss = (prediction - labels).sum()
loss.backward()

In [5]:
# load optimizer, SGD learning rate=0.01 and momentum = 0.9
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

optim.step() # Initiate gradient descent. optimizer adjusts each parameter by its gradient

# Differentiation in Autograd

In [6]:
import torch

a = torch.tensor([2., 3.], requires_grad=True) # True means now all operation will be tracked on this tensor.
b = torch.tensor([6., 4.], requires_grad=True) 

create another tensor $$Q = 3a^3 - b^2$$

In [7]:
Q = 3*a**3 - b**2

Q is an error and a and b are 2 parameters of NN.

$$\frac{\partial Q}{\partial a} = 9a^2$$

$$\frac{\partial Q}{\partial b} = -2b$$

When we call backward() on Q, autograd calculates these gradients and stores them in the respective tensor's `.grad` attribute.

We need to explicitly pass a gradient argument in Q.backward() because it is a vector. gradient is a tensor of the same shape as Q

In [8]:
external_grad = torch.tensor([1.,1.])
Q.backward(gradient=external_grad)

# Gradients are now deposited in a.grad and b.grad

print(a.grad)
print(b.grad)

tensor([36., 81.])
tensor([-12.,  -8.])


Optional Reading - Vector Calculus using `autograd`
===================================================

Mathematically, if you have a vector valued function
$\vec{y}=f(\vec{x})$, then the gradient of $\vec{y}$ with respect to
$\vec{x}$ is a Jacobian matrix $J$:

$$\begin{aligned}
J
=
 \left(\begin{array}{cc}
 \frac{\partial \bf{y}}{\partial x_{1}} &
 ... &
 \frac{\partial \bf{y}}{\partial x_{n}}
 \end{array}\right)
=
\left(\begin{array}{ccc}
 \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
 \vdots & \ddots & \vdots\\
 \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
 \end{array}\right)
\end{aligned}$$

Generally speaking, `torch.autograd` is an engine for computing
vector-Jacobian product. That is, given any vector $\vec{v}$, compute
the product $J^{T}\cdot \vec{v}$

If $\vec{v}$ happens to be the gradient of a scalar function
$l=g\left(\vec{y}\right)$:

$$\vec{v}
 =
 \left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$$

then by the chain rule, the vector-Jacobian product would be the
gradient of $l$ with respect to $\vec{x}$:

$$\begin{aligned}
J^{T}\cdot \vec{v}=\left(\begin{array}{ccc}
 \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
 \vdots & \ddots & \vdots\\
 \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
 \end{array}\right)\left(\begin{array}{c}
 \frac{\partial l}{\partial y_{1}}\\
 \vdots\\
 \frac{\partial l}{\partial y_{m}}
 \end{array}\right)=\left(\begin{array}{c}
 \frac{\partial l}{\partial x_{1}}\\
 \vdots\\
 \frac{\partial l}{\partial x_{n}}
 \end{array}\right)
\end{aligned}$$

This characteristic of vector-Jacobian product is what we use in the
above example; `external_grad` represents $\vec{v}$.


Computational Graph
===================

Conceptually, autograd keeps a record of data (tensors) & all executed
operations (along with the resulting new tensors) in a directed acyclic
graph (DAG) consisting of
[Function](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
objects. In this DAG, leaves are the input tensors, roots are the output
tensors. By tracing this graph from roots to leaves, you can
automatically compute the gradients using the chain rule.

In a forward pass, autograd does two things simultaneously:

-   run the requested operation to compute a resulting tensor, and
-   maintain the operation's *gradient function* in the DAG.

The backward pass kicks off when `.backward()` is called on the DAG
root. `autograd` then:

-   computes the gradients from each `.grad_fn`,
-   accumulates them in the respective tensor's `.grad` attribute, and
-   using the chain rule, propagates all the way to the leaf tensors.

Below is a visual representation of the DAG in our example. In the
graph, the arrows are in the direction of the forward pass. The nodes
represent the backward functions of each operation in the forward pass.
The leaf nodes in blue represent our leaf tensors `a` and `b`.

![](https://pytorch.org/tutorials/_static/img/dag_autograd.png)

<div style="background-color: #54c7ec; color: #fff; font-weight: 700; padding-left: 10px; padding-top: 5px; padding-bottom: 5px"><strong>NOTE:</strong></div>
<div style="background-color: #f3f4f7; padding-left: 10px; padding-top: 10px; padding-bottom: 10px; padding-right: 10px">
<p>An important thing to note is that the graph is recreated from scratch; after each<code>.backward()</code> call, autograd starts populating a new graph. This isexactly what allows you to use control flow statements in your model;you can change the shape, size and operations at every iteration ifneeded.</p>
</div>

Exclusion from the DAG
----------------------

`torch.autograd` tracks operations on all tensors which have their
`requires_grad` flag set to `True`. For tensors that don't require
gradients, setting this attribute to `False` excludes it from the
gradient computation DAG.

The output tensor of an operation will require gradients even if only a
single input tensor has `requires_grad=True`.


In [9]:
# In a NN, parameters that don’t compute gradients are usually called frozen parameters.
# Let's freeze all parameters.

from torch import nn, optim
model = resnet18(weights=ResNet18_Weights.DEFAULT)

for param in model.parameters():
    param.requires_grad = False

In [10]:
# we are replacing last linear layer model.fc with new linear layer. So, new layer is unfrozen.
model.fc = nn.Linear(512,10) # 10 labels.
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)